# Step 2. Process Data

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import time
from datetime import datetime
import glob

import json
from tqdm import tqdm

import pandas as pd
pd.set_option('display.max_rows', 50)

In [ ]:
data_dir = '../data'
os.makedirs(data_dir, exist_ok=True)

In [ ]:
with open(f'{data_dir}/months.json') as f:
    months = json.load(f)

df_months = pd.DataFrame(months)
print(df_months.shape)
df_months.head(3)

In [ ]:
with open(f'{data_dir}/states.json') as f:
    states = json.load(f)

df_states = pd.DataFrame(states)
print(df_states.shape)
df_states.head(3)

In [ ]:
with open(f'{data_dir}/districts.json') as f:
    districts = json.load(f)

df_districts = []
for state in districts:
    for district in state['districts']:
        district['state_id'] = state['state']['value']
        district['state_name'] = state['state']['name']
        district['district_id'] = district.pop('value')
        district['district_name'] = district.pop('name')
        df_districts.append(district)

df_districts = pd.DataFrame(df_districts)

# Remove placeholder rows.
df_districts = df_districts[(df_districts['district_id'] != '0') & (df_districts['state_id'] != '0')] 
df_districts.sort_values(by=['state_id', 'district_id'], inplace=True)

print(df_districts.shape)
df_districts[df_districts['state_name'] == 'Karnataka']

In [ ]:
def parse_filepath(filepath):
    parts = os.path.normpath(filepath).split(os.sep)
    filename = parts[-1]

    if len(parts) >= 2 and parts[-2].isdigit():
        state_id = int(parts[-2])
    else:
        state_id = None

    match = re.match(r"(\d{4})_(\d{1,2})\.json", filename)
    if match:
        year, month = map(int, match.groups())
        return {
            "state_id": state_id,
            "year": year,
            "month": month
        }
    else:
        raise ValueError(f"Filename {filename} does not match expected pattern")

In [ ]:
def flatten_record(record):
    flat = {'district_id': record['district_id']}
    for entry in record['data']:
        flat[entry['title']] = entry['count']
    return flat

In [ ]:
def process_dashboard(filepath):

    with open(filepath) as f:
        data = json.load(f)

    records = []
    for item in data['registrationPTdata']:
        for district in item['districts']:
            record = flatten_record(district)

            parts = parse_filepath(filepath)
            record.update(parts)
            
            records.append(record)
    return pd.DataFrame(records)

In [ ]:
def clean_column_name(col):
    col = col.lower()
    col = re.sub(r'[^a-z0-9]+', '_', col)
    col = col.strip('_')
    return col

In [ ]:
files = glob.glob('../data/dashboard/**/*.json', recursive=True)
print('Number of files:', len(files), '\n')

records = []
for filepath in files:
    records.append(process_dashboard(filepath))

df_dashboard = pd.concat(records)

df_dashboard.columns = [clean_column_name(c) for c in df_dashboard.columns]
df_dashboard = df_dashboard.reindex(sorted(df_dashboard.columns), axis=1)

print(df_dashboard.shape)
df_dashboard.head(3)

In [ ]:
# Handle typo in the column name: Angwanwadi
# Ref: https://github.com/aiforhealth-xyz/poshan-tracker/discussions/1
df_dashboard['anganwadi_centers'] = df_dashboard.apply(
    lambda row: row['anganwadi_centers'] if pd.notna(row['anganwadi_centers']) else row['angwanwadi_centers'],
    axis=1
)
df_dashboard = df_dashboard.drop(columns=['angwanwadi_centers'])

In [ ]:
df_dashboard.to_csv(f'{data_dir}/dashboard.csv', index=False)

print(df_dashboard.shape)
df_dashboard.head(3)